In [ ]:
%reload_ext nb_black

In [21]:
# creating supervised learning imports

import pandas as pd

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest

from sklearn.metrics import (
    classification_report,
    confusion_matrix,
)

<IPython.core.display.Javascript object>

In [3]:
final_df = pd.read_csv("./cleaned_data.csv")
final_df.head()

,event_id,app_id,is_active,device_id,group,8,9,10,11,12,...,device_model_魅蓝,device_model_魅蓝2,device_model_魅蓝NOTE,device_model_魅蓝Note 2,device_model_魅蓝metal,device_model_麦芒3,device_model_麦芒3S,device_model_麦芒4,device_model_黄金斗士A8,device_model_黄金斗士Note8
0,6,-8764672938472212518,1,1476664663289716375,M22-,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,6,-8271866350659046570,0,1476664663289716375,M22-,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,-7509752927626140732,0,1476664663289716375,M22-,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,6,-7377004479023402858,1,1476664663289716375,M22-,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,6,-5839858269967688123,0,1476664663289716375,M22-,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


<IPython.core.display.Javascript object>

In [4]:
# Need to convert these values to numeric values
# Considering:
# M22- : 0
# M23-26 : 1
# M27-28 : 2
# M29-31 : 3
# M32-38 : 4
# M39+ : 5
# F23- : 6
# F24-26 : 7
# F27-28 : 8
# F29-32 : 9
# F33-42 : 10
# F43+ : 11
final_df["group"].value_counts()

M32-38    80954
M39+      71960
M29-31    55421
M23-26    50400
M27-28    33676
F33-42    30293
M22-      28706
F43+      23410
F29-32    20850
F23-      13852
F27-28    13659
F24-26    12809
Name: group, dtype: int64

<IPython.core.display.Javascript object>

In [5]:
# Converting object data to numeric data in group column
to_num = {
    "M22-": 0,
    "M23-26": 1,
    "M27-28": 2,
    "M29-31": 3,
    "M32-38": 4,
    "M39+": 5,
    "F23-": 6,
    "F24-26": 7,
    "F27-28": 8,
    "F29-32": 9,
    "F33-42": 10,
    "F43+": 11,
}

# Created new column for numeric group, can now drop old group column
final_df["num_group"] = final_df["group"].map(to_num)
final_df = final_df.drop(columns=["group"])
final_df.head()

,event_id,app_id,is_active,device_id,8,9,10,11,12,13,...,device_model_魅蓝2,device_model_魅蓝NOTE,device_model_魅蓝Note 2,device_model_魅蓝metal,device_model_麦芒3,device_model_麦芒3S,device_model_麦芒4,device_model_黄金斗士A8,device_model_黄金斗士Note8,num_group
0,6,-8764672938472212518,1,1476664663289716375,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,6,-8271866350659046570,0,1476664663289716375,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,-7509752927626140732,0,1476664663289716375,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,6,-7377004479023402858,1,1476664663289716375,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,6,-5839858269967688123,0,1476664663289716375,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


<IPython.core.display.Javascript object>

In [6]:
num_cols = ["app_id", "device_id", "event_id"]

<IPython.core.display.Javascript object>

In [7]:
# splitting up the data and choosing num_group as target variable
X = final_df.drop(columns=["num_group"])
y = final_df["num_group"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=13, stratify=y
)

<IPython.core.display.Javascript object>

In [8]:
# set up preprocessing for pipeline (used to scale features)
preprocessing = ColumnTransformer(
    [("scale", StandardScaler(), num_cols),], remainder="passthrough",
)

<IPython.core.display.Javascript object>

# Original Models

### KNearestNeighbor Classifier

In [9]:
pipeline = Pipeline([("preprocessing", preprocessing), ("knn", KNeighborsClassifier())])

pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('scale', StandardScaler(),
                                                  ['app_id', 'device_id',
                                                   'event_id'])])),
                ('knn', KNeighborsClassifier())])

<IPython.core.display.Javascript object>

In [10]:
grid = {
    "knn__n_neighbors": [3, 5, 7, 9, 11,],
    "knn__weights": ["distance", "uniform"],
}

model = GridSearchCV(pipeline, grid, n_jobs=-1, verbose=1)
model.fit(X_train, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed: 271.0min
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed: 517.6min finished


GridSearchCV(estimator=Pipeline(steps=[('preprocessing',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('scale',
                                                                         StandardScaler(),
                                                                         ['app_id',
                                                                          'device_id',
                                                                          'event_id'])])),
                                       ('knn', KNeighborsClassifier())]),
             n_jobs=-1,
             param_grid={'knn__n_neighbors': [3, 5, 7, 9, 11],
                         'knn__weights': ['distance', 'uniform']},
             verbose=1)

<IPython.core.display.Javascript object>

In [11]:
# Best params come from highest amount of n_neighbors, will increase this value for next fitting
model.best_params_

{'knn__n_neighbors': 11, 'knn__weights': 'distance'}

<IPython.core.display.Javascript object>

In [ ]:
# Model appears to be extremely overfitting
train_score = model.score(X_train, y_train)
test_score = model.score(X_test, y_test)

print(f"Train score: {train_score}")
print(f"Test score: {test_score}")

In [13]:
y_pred = model.predict(X_test)

con_mat = pd.DataFrame(
    confusion_matrix(y_test, y_pred),
    index=[
        "Actually M22-",
        "Actually M23-26",
        "Actually M27-28",
        "Actually M29-31",
        "Actually M32-38",
        "Actually M39+",
        "Actually F23-",
        "Actually F24-26",
        "Actually F27-28",
        "Actually F29-32",
        "Actually F33-42",
        "Actually F43+",
    ],
    columns=[
        "Predicted M22-",
        "Predicted M23-26",
        "Predicted M27-28",
        "Predicted M29-31",
        "Predicted M32-38",
        "Predicted M39+",
        "Predicted F23-",
        "Predicted F24-26",
        "Predicted F27-28",
        "Predicted F29-32",
        "Predicted F33-42",
        "Predicted F43+",
    ],
)
con_mat.style.background_gradient(axis=None)

,Predicted M22-,Predicted M23-26,Predicted M27-28,Predicted M29-31,Predicted M32-38,Predicted M39+,Predicted F23-,Predicted F24-26,Predicted F27-28,Predicted F29-32,Predicted F33-42,Predicted F43+
Actually M22-,1249,808,356,695,1042,757,183,86,99,146,199,121
Actually M23-26,713,2354,613,1264,2256,1473,199,150,151,240,400,267
Actually M27-28,417,770,1022,941,1541,1106,104,92,85,196,275,186
Actually M29-31,516,1194,688,2689,2549,2027,149,144,147,224,481,276
Actually M32-38,609,1644,862,1855,5786,3122,220,170,251,422,765,485
Actually M39+,463,1072,595,1507,3312,5436,140,134,193,353,752,435
Actually F23-,279,371,141,288,490,323,420,85,55,91,161,66
Actually F24-26,197,321,134,299,481,373,111,220,82,117,145,82
Actually F27-28,136,332,144,290,622,455,63,63,249,104,192,82
Actually F29-32,210,423,228,439,968,762,108,78,84,443,260,167


<IPython.core.display.Javascript object>

In [14]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.24      0.22      0.23      5741
           1       0.23      0.23      0.23     10080
           2       0.19      0.15      0.17      6735
           3       0.24      0.24      0.24     11084
           4       0.27      0.36      0.31     16191
           5       0.30      0.38      0.33     14392
           6       0.22      0.15      0.18      2770
           7       0.16      0.09      0.11      2562
           8       0.16      0.09      0.12      2732
           9       0.16      0.11      0.13      4170
          10       0.19      0.15      0.16      6059
          11       0.19      0.12      0.14      4682

    accuracy                           0.24     87198
   macro avg       0.21      0.19      0.20     87198
weighted avg       0.23      0.24      0.24     87198



<IPython.core.display.Javascript object>

### Random Forest Classifier

In [22]:
pipeline = Pipeline(
    [("preprocessing", preprocessing), ("rf", RandomForestClassifier())]
)

pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('scale', StandardScaler(),
                                                  ['app_id', 'device_id',
                                                   'event_id'])])),
                ('rf', RandomForestClassifier())])

<IPython.core.display.Javascript object>

In [23]:
grid = {
    "rf__max_depth": [3, 5, 7, 10, 15],
    "rf__n_estimators": [1, 10, 100, 1000],
    "rf__min_samples_leaf": [1, 3, 5, 7, 10],
    "rf__criterion": ["gini", "entropy"],
}

model = GridSearchCV(pipeline, grid, n_jobs=-1, verbose=2)
model.fit(X_train, y_train)

Fitting 5 folds for each of 200 candidates, totalling 1000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done 138 tasks      | elapsed: 27.2min
[Parallel(n_jobs=-1)]: Done 341 tasks      | elapsed: 98.8min
[Parallel(n_jobs=-1)]: Done 624 tasks      | elapsed: 242.0min
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed: 458.6min finished


GridSearchCV(estimator=Pipeline(steps=[('preprocessing',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('scale',
                                                                         StandardScaler(),
                                                                         ['app_id',
                                                                          'device_id',
                                                                          'event_id'])])),
                                       ('rf', RandomForestClassifier())]),
             n_jobs=-1,
             param_grid={'rf__criterion': ['gini', 'entropy'],
                         'rf__max_depth': [3, 5, 7, 10, 15],
                         'rf__min_samples_leaf': [1, 3, 5, 7, 10],
                         'rf__n_estimators': [1, 10, 100, 1000]},
             verbose=2)

<IPython.core.display.Javascript object>

In [24]:
model.best_params_

{'rf__criterion': 'gini',
 'rf__max_depth': 15,
 'rf__min_samples_leaf': 1,
 'rf__n_estimators': 100}

<IPython.core.display.Javascript object>

In [25]:
train_score = model.score(X_train, y_train)
test_score = model.score(X_test, y_test)

print(f"Train score: {train_score}")
print(f"Test score: {test_score}")

Train score: 0.2647681139475676
Test score: 0.2436753136539829


<IPython.core.display.Javascript object>

In [26]:
y_pred = model.predict(X_test)

con_mat = pd.DataFrame(
    confusion_matrix(y_test, y_pred),
    index=[
        "Actually M22-",
        "Actually M23-26",
        "Actually M27-28",
        "Actually M29-31",
        "Actually M32-38",
        "Actually M39+",
        "Actually F23-",
        "Actually F24-26",
        "Actually F27-28",
        "Actually F29-32",
        "Actually F33-42",
        "Actually F43+",
    ],
    columns=[
        "Predicted M22-",
        "Predicted M23-26",
        "Predicted M27-28",
        "Predicted M29-31",
        "Predicted M32-38",
        "Predicted M39+",
        "Predicted F23-",
        "Predicted F24-26",
        "Predicted F27-28",
        "Predicted F29-32",
        "Predicted F33-42",
        "Predicted F43+",
    ],
)
con_mat.style.background_gradient(axis=None)

,Predicted M22-,Predicted M23-26,Predicted M27-28,Predicted M29-31,Predicted M32-38,Predicted M39+,Predicted F23-,Predicted F24-26,Predicted F27-28,Predicted F29-32,Predicted F33-42,Predicted F43+
Actually M22-,216,714,2,103,4049,641,16,0,0,0,0,0
Actually M23-26,21,1356,6,124,7184,1375,11,2,0,1,0,0
Actually M27-28,3,392,44,147,5025,1122,2,0,0,0,0,0
Actually M29-31,15,495,9,506,7964,2094,1,0,0,0,0,0
Actually M32-38,5,437,1,43,12740,2960,1,0,0,0,2,2
Actually M39+,14,289,3,40,7798,6246,1,0,0,0,1,0
Actually F23-,13,254,2,31,2147,246,77,0,0,0,0,0
Actually F24-26,18,201,1,5,1977,320,21,18,1,0,0,0
Actually F27-28,6,128,0,24,2162,392,13,0,6,0,1,0
Actually F29-32,4,161,3,14,3113,850,4,0,1,15,5,0


<IPython.core.display.Javascript object>

In [ ]:
print(classification_report(y_test, y_pred))

### Support Vector Classifier

In [ ]:
pipeline = Pipeline([("preprocessing", preprocessing), ("SVC", SVC())])

pipeline.fit(X_train, y_train)

In [ ]:
grid = {
    "SVC__kernel": ["linear", "rbf", "poly"],
    "SVC__C": [1, 10, 100, 1000],
    "SVC__degree": [3, 5, 7, 10],
}

model = GridSearchCV(pipeline, grid, n_jobs=-1,verbose = 2)
model.fit(X_train, y_train)

In [ ]:
model.best_params_

In [ ]:
train_score = model.score(X_train, y_train)
test_score = model.score(X_test, y_test)

print(f"Train score: {train_score}")
print(f"Test score: {test_score}")

In [ ]:
y_pred = model.predict(X_test)

con_mat = pd.DataFrame(
    confusion_matrix(y_test, y_pred),
    index=[
        "Actually M22-",
        "Actually M23-26",
        "Actually M27-28",
        "Actually M29-31",
        "Actually M32-38",
        "Actually M39+",
        "Actually F23-",
        "Actually F24-26",
        "Actually F27-28",
        "Actually F29-32",
        "Actually F33-42",
        "Actually F43+",
    ],
    columns=[
        "Predicted M22-",
        "Predicted M23-26",
        "Predicted M27-28",
        "Predicted M29-31",
        "Predicted M32-38",
        "Predicted M39+",
        "Predicted F23-",
        "Predicted F24-26",
        "Predicted F27-28",
        "Predicted F29-32",
        "Predicted F33-42",
        "Predicted F43+",
    ],
)
con_mat.style.background_gradient(axis=None)

In [ ]:
print(classification_report(y_test, y_pred))

# Models with PCA

In [ ]:
# Since I am performing pca its necessary to standardize data and fit it to data frame.
scaler = StandardScaler()
X_train_std = scaler.fit_transform(X_train)

# Computing the principal components
pca = PCA()
X_pca = pca.fit_transform(X_train_std)

In [ ]:
# I plan to use the amount of components that add up to 90% variance.
n_components = np.sum(np.cumsum(pca.explained_variance_ratio_) < 0.90)
X_pca = X_pca[:, :n_components]

In [ ]:
# check to see the amount of components being used
n_components

### KNearestNeighbor Classifier

In [ ]:
pipeline = Pipeline(
    [("scaler", StandardScaler()), ("pca", PCA(n_components = n_components)), ("knn", KNeighborsClassifier()),]
)
pipeline = pipeline.fit(X_train, y_train)

In [ ]:
grid = {
    "knn__n_neighbors": [3, 5, 7, 9, 11,],
    "knn__weights": ["distance", "uniform"],
}

model = GridSearchCV(pipeline, grid, n_jobs=-1, verbose=2)
model.fit(X_train, y_train)

In [ ]:
model.best_params_

In [ ]:
train_score = model.score(X_train, y_train)
test_score = model.score(X_test, y_test)

print(f"Train score: {train_score}")
print(f"Test score: {test_score}")

In [ ]:
y_pred = model.predict(X_test)

con_mat = pd.DataFrame(
    confusion_matrix(y_test, y_pred),
    index=[
        "Actually M22-",
        "Actually M23-26",
        "Actually M27-28",
        "Actually M29-31",
        "Actually M32-38",
        "Actually M39+",
        "Actually F23-",
        "Actually F24-26",
        "Actually F27-28",
        "Actually F29-32",
        "Actually F33-42",
        "Actually F43+",
    ],
    columns=[
        "Predicted M22-",
        "Predicted M23-26",
        "Predicted M27-28",
        "Predicted M29-31",
        "Predicted M32-38",
        "Predicted M39+",
        "Predicted F23-",
        "Predicted F24-26",
        "Predicted F27-28",
        "Predicted F29-32",
        "Predicted F33-42",
        "Predicted F43+",
    ],
)
con_mat.style.background_gradient(axis=None)

In [ ]:
print(classification_report(y_test, y_pred))

### Random Forest Classifier

In [ ]:
pipeline = Pipeline(
    [("scaler", StandardScaler()), ("pca", PCA(n_components = n_components)), ("rf", RandomForestClassifier()),]
)
pipeline = pipeline.fit(X_train, y_train)

In [ ]:
grid = {
    "rf__max_depth": [3, 5, 7, 10, 15],
    "rf__n_estimators": [1, 10, 100, 1000],
    "rf__min_samples_leaf": [1, 3, 5, 7, 10],
    "rf__criterion": ["gini", "entropy"],
}

model = GridSearchCV(pipeline, grid, n_jobs=-1,verbose = 2)
model.fit(X_train, y_train)

In [ ]:
model.best_params_

In [ ]:
train_score = model.score(X_train, y_train)
test_score = model.score(X_test, y_test)

print(f"Train score: {train_score}")
print(f"Test score: {test_score}")

In [ ]:
y_pred = model.predict(X_test)

con_mat = pd.DataFrame(
    confusion_matrix(y_test, y_pred),
    index=[
        "Actually M22-",
        "Actually M23-26",
        "Actually M27-28",
        "Actually M29-31",
        "Actually M32-38",
        "Actually M39+",
        "Actually F23-",
        "Actually F24-26",
        "Actually F27-28",
        "Actually F29-32",
        "Actually F33-42",
        "Actually F43+",
    ],
    columns=[
        "Predicted M22-",
        "Predicted M23-26",
        "Predicted M27-28",
        "Predicted M29-31",
        "Predicted M32-38",
        "Predicted M39+",
        "Predicted F23-",
        "Predicted F24-26",
        "Predicted F27-28",
        "Predicted F29-32",
        "Predicted F33-42",
        "Predicted F43+",
    ],
)
con_mat.style.background_gradient(axis=None)

In [ ]:
print(classification_report(y_test, y_pred))

### Support Vector Classifier

In [ ]:
pipeline = Pipeline(
    [("scaler", StandardScaler()), ("pca", PCA(n_components = n_components)), ("SVC", SVC()),]
)
pipeline = pipeline.fit(X_train, y_train)

In [ ]:
grid = {
    "SVC__kernel": ["linear", "rbf", "poly"],
    "SVC__C": [1, 10, 100, 1000],
    "SVC__degree": [3, 5, 7, 10],
}

model = GridSearchCV(pipeline, grid, n_jobs=-1,verbose = 2)
model.fit(X_train, y_train)

In [ ]:
model.best_params_

In [ ]:
train_score = model.score(X_train, y_train)
test_score = model.score(X_test, y_test)

print(f"Train score: {train_score}")
print(f"Test score: {test_score}")

In [ ]:
y_pred = model.predict(X_test)

con_mat = pd.DataFrame(
    confusion_matrix(y_test, y_pred),
    index=[
        "Actually M22-",
        "Actually M23-26",
        "Actually M27-28",
        "Actually M29-31",
        "Actually M32-38",
        "Actually M39+",
        "Actually F23-",
        "Actually F24-26",
        "Actually F27-28",
        "Actually F29-32",
        "Actually F33-42",
        "Actually F43+",
    ],
    columns=[
        "Predicted M22-",
        "Predicted M23-26",
        "Predicted M27-28",
        "Predicted M29-31",
        "Predicted M32-38",
        "Predicted M39+",
        "Predicted F23-",
        "Predicted F24-26",
        "Predicted F27-28",
        "Predicted F29-32",
        "Predicted F33-42",
        "Predicted F43+",
    ],
)
con_mat.style.background_gradient(axis=None)

In [ ]:
print(classification_report(y_test, y_pred))

# Models with SelectKBest

### KNearestNeighbor Classifier